In [2]:
import numpy as np
import pandas as pd

In [245]:
#КОЛЬЦА
#загружаем две таблицы с приходами по всем ТГ 4001 и 4093
opt_prih=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\ифмарт1.xlsx',
    sheet_name='4093',usecols=[4,0,9,11],names=['Производитель','Тип 1','Номер','Приходы_4093'])
cs_prih=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\ифмарт1.xlsx',
    sheet_name='4001',usecols=[4,0,9,11],names=['Производитель','Тип 1','Номер','Приходы_4001'])

#соединием 2 таблицы с приходами 4001 и 4093 в одну
prih_merged=pd.merge(opt_prih,cs_prih,how = 'outer')

#загружаем две таблицы остатки КОЛЬЦА 4001 и 4093 и одну продажи
balance=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\ифмарт1.xlsx',
    sheet_name='ост_к',skiprows=9) #использовать sheet_name='ост' для неколец
balance_4093=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\ифмарт1.xlsx',
    sheet_name='ост_4093',skiprows=9,names=['Номер','Остаток_4093'])
sales=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\ифмарт1.xlsx',
    sheet_name='прод',skiprows=12)

#СОЕДИНЯЕМ ПРОДАЖИ, ОСТАТКИ 4093 И ОСТАТКИ В ОДНУ ТАБЛИЦУ
merged=pd.merge(balance,balance_4093,on='Номер',how = 'left')
del sales['Тип 1'],sales['Производитель']
merged=pd.merge(merged,sales,on='Номер',how = 'left',suffixes=('_ос', '_пр'))

# фильтр приходов по кольцам
prih_merged=prih_merged[
    (prih_merged['Тип 1'] == 'КОЛЬЦО')|
    (prih_merged['Тип 1'] == 'КОЛЬЦО ОБРУЧАЛЬНОЕ')|
    (prih_merged['Тип 1'] == 'КОЛЬЦО ПЕЧАТКА')
]

#соединяем приходы, остатки и продажи
merged=pd.merge(prih_merged,merged,how = 'outer')

# добавляем в финальную таблицу недостающую инфу по карточкам
items=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\ифмарт1.xlsx',
    sheet_name='карточки',skiprows=6)
del items['Описание 3']

final=pd.merge(merged,items,on='Номер',how = 'left')

#удаляем опт, ИМ
final=final[~(final['Группа наценки'].str.contains('опт',case=False,na=False))]#na=False
final=final[~(final['Артикул товара'].str.contains('#ИМ',case=True,na=False))]#na=False

col_names=final.columns.tolist()

#Суммируем приходы, продажи остатки чтобы и убираем товы с нулями
#final['сумма']=final.groupby(by='Артикул товара')[col_names[5:7]].sum(axis=1)

#выстраиваем столбцы в нужном порядке
columns=[col_names[18],col_names[1],col_names[0]]+col_names[19:21]+[col_names[2]]+col_names[21:25]+col_names[12:18]+col_names[5:12]+col_names[3:5]+col_names[-5:]
final=final[columns]
#final=final.set_index('Артикул товара',drop=True, inplace=True)

final.to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\иф_кольца.xlsx')

In [14]:
#НЕКОЛЬЦА
#загружаем две таблицы с приходами по всем ТГ 4001 и 4093
opt_prih=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\ифмарт1.xlsx',
    sheet_name='4093',usecols=[4,0,9,11],names=['Производитель','Тип 1','Номер','Приходы_4093'])
cs_prih=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\ифмарт1.xlsx',
    sheet_name='4001',usecols=[4,0,9,11],names=['Производитель','Тип 1','Номер','Приходы_4001'])

#соединием 2 таблицы с приходами 4001 и 4093 в одну
prih_merged=pd.merge(opt_prih,cs_prih,how = 'outer')

#загружаем две таблицы остатки КОЛЬЦА 4001 и 4093 и одну продажи
balance=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\ифмарт1.xlsx',
    sheet_name='ост',skiprows=9) #использовать sheet_name='ост' для неколец
balance_4093=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\ифмарт1.xlsx',
    sheet_name='ост_4093',skiprows=9,names=['Номер','Остаток_4093'])
sales=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\ифмарт1.xlsx',
    sheet_name='прод',skiprows=12)

#СОЕДИНЯЕМ ПРОДАЖИ, ОСТАТКИ 4093 И ОСТАТКИ В ОДНУ ТАБЛИЦУ
merged=pd.merge(balance,balance_4093,on='Номер',how = 'left')
del sales['Тип 1'],sales['Производитель']
merged=pd.merge(merged,sales,on='Номер',how = 'left',suffixes=('_ос', '_пр'))
# фильтр приходов по кольцам
prih_merged=prih_merged[
    (prih_merged['Тип 1'] != 'КОЛЬЦО')&
    (prih_merged['Тип 1'] != 'КОЛЬЦО ОБРУЧАЛЬНОЕ')&
    (prih_merged['Тип 1'] != 'КОЛЬЦО ПЕЧАТКА')
]

#соединяем приходы, остатки и продажи
merged=pd.merge(prih_merged,merged,how = 'outer')

# добавляем в финальную таблицу недостающую инфу по карточкам
items=pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\ифмарт1.xlsx',
    sheet_name='карточки',skiprows=6)
del items['Размер']

final=pd.merge(merged,items,on='Номер',how = 'left')

#удаляем опт, ИМ
final=final[~(final['Группа наценки'].str.contains('опт',case=False,na=False))]
final=final[~(final['Артикул товара'].str.contains('#ИМ',case=True,na=False))]

#выстраиваем столбцы в нужном порядке
col_names=final.columns.tolist()
columns=[col_names[18],col_names[1],col_names[0]]+col_names[19:21]+[col_names[2]]+col_names[21:25]+col_names[12:18]+col_names[5:12]+col_names[3:5]+col_names[-5:]
final=final[columns]

#Суммируем приходы, продажи остатки чтобы и убираем товы с нулями
final['сумма']=final[col_names[3:18]].sum(axis=1)
final=final[final['сумма'] >0]
del final['сумма']

#Удаляем буквы и зодиаки
final=final[(final['Дизайн'] !='ИФ БУКВЫ') & (final['Дизайн'] !='ИФ ЗОДИАК')]

final.to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\иф_некольца.xlsx')

,Артикул товара,Тип 1,Производитель,Товарная подгруппа,Ценовая корзина,Номер,Описание,Дизайн,Размер,Количество камней,...,"Февраль,2019_ос",Остаток_4093,Приходы_4093,Приходы_4001,Средний вес изделия,Тип 3,Вставка камней,Фото изделия,Тип изготовления,сумма
0,12-00-0001-20536,СЕРЬГИ,ООО «ЮИ-ТРЕЙД»,1040.0,ДЕШЕВАЯ,ТОВ0690909,"Серьги, фианит",ИФ ПУССЕТЫ ОДИНОЧКА Б/Ц,0.0,2.0,...,3.0,NaN,100.0,NaN,1.08,КРАСНЫЙ,ФИАНИТ БЕСЦВЕТ,\\gold585.int\uk\общее хранилище файлов\Коммер...,NaN,142.0
1,50-03-0001-20662,ПОДВЕС ДЕКОРАТИВНЫЙ,ООО «ЮИ-ТРЕЙД»,1067.0,ДЕШЕВАЯ,ТОВ0691019,"Подвес, фианит",ИФ ПЕТЛИ,0.0,2.0,...,24.0,NaN,150.0,NaN,0.81,КРАСНЫЙ,ФИАНИТ БЕСЦВЕТ,\\gold585.int\uk\общее хранилище файлов\Коммер...,NaN,336.0
2,10-02-0001-21338,СЕРЬГИ,ООО «ЮИ-ТРЕЙД»,1025.0,ДЕШЕВАЯ,ТОВ0751210,"Серьги, фианит",ИФ ЛЕПЕСТКИ,0.0,14.0,...,219.0,0.0,200.0,NaN,1.74,КРАСНЫЙ,ФИАНИТ БЕСЦВЕТ,\\gold585.int\uk\общее хранилище файлов\Коммер...,NaN,1958.0
3,10-02-0001-21497,СЕРЬГИ,ООО «ЮИ-ТРЕЙД»,1031.0,ДЕШЕВАЯ,ТОВ0751212,"Серьги, фианит",ИФ ДОРОЖКА,0.0,24.0,...,57.0,NaN,50.0,NaN,1.85,КРАСНЫЙ,ФИАНИТ БЕСЦВЕТ,\\gold585.int\uk\общее хранилище файлов\Коммер...,NaN,512.0
4,50-02-0001-20774,ПОДВЕС ДЕКОРАТИВНЫЙ,ООО «ЮИ-ТРЕЙД»,1068.0,СРЕДНЯЯ,ТОВ0751227,"Подвес, фианит",ИФ ПЕТЛИ,0.0,32.0,...,62.0,NaN,200.0,NaN,1.27,КРАСНЫЙ,ФИАНИТ БЕСЦВЕТ,\\gold585.int\uk\общее хранилище файлов\Коммер...,NaN,712.0
5,13-02-0001-21910#$$,СЕРЬГИ,ООО «ЮИ-ТРЕЙД»,1048.0,ДОРОГАЯ,ТОВ0751453,"Серьги, фианит",ИФ ДЕТСКИЕ Б/Ц,0.0,62.0,...,41.0,29.0,21.0,NaN,1.58,КРАСНЫЙ,ФИАНИТ БЕСЦВЕТ,\\gold585.int\uk\общее хранилище файлов\Коммер...,NaN,175.0
6,ПС 001,СЕРЬГИ,"ООО ""Талант""",1045.0,ДОРОГАЯ,ТОВ0756896,"Серьги, кубический цирконий",ИФ ПУССЕТЫ ПРОЧЕЕ ЦВЕТН,0.0,2.0,...,348.0,1.0,450.0,NaN,1.61,КРАСНЫЙ,ФИАНИТ БЕСЦВЕТ,\\gold585.int\uk\общее хранилище файлов\Коммер...,NaN,3948.0
7,КР 002,ПОДВЕС КУЛЬТОВЫЙ,"ООО ""Талант""",1061.0,ДЕШЕВАЯ,ТОВ0756900,"Крест, кубический цирконий",ИФ КРЕСТ,0.0,11.0,...,412.0,0.0,250.0,NaN,0.54,КРАСНЫЙ,ФИАНИТ БЕСЦВЕТ,\\gold585.int\uk\общее хранилище файлов\Коммер...,NaN,3820.0
8,СД 001-70,СЕРЬГИ,"ООО ""Талант""",1046.0,ДЕШЕВАЯ,ТОВ0760004,"Серьги, кубический цирконий",ИФ ДЕТСКИЕ Б/Ц,0.0,2.0,...,328.0,0.0,500.0,NaN,1.04,КРАСНЫЙ,ФИАНИТ БЕСЦВЕТ,\\gold585.int\uk\общее хранилище файлов\Коммер...,NaN,3293.0
9,10-02-0001-20532,СЕРЬГИ,ООО «ЮИ-ТРЕЙД»,1032.0,СРЕДНЯЯ,ТОВ0762245,"Серьги, фианит",ИФ МНОГОКАМЕНКА,0.0,80.0,...,534.0,14.0,10.0,NaN,2.27,КРАСНЫЙ,ФИАНИТ БЕСЦВЕТ,\\gold585.int\uk\общее хранилище файлов\Коммер...,NaN,2778.0


In [15]:
items

,Номер,Артикул товара,Товарная подгруппа,Ценовая корзина,Описание,Дизайн,Размер,Количество камней,Группа наценки,Средний вес изделия,Тип 3,Вставка камней,Фото изделия,Тип изготовления
0,ТОВ0121160,01Ш010003,1091,ДЕШЕВАЯ,"Брошь, фианит",NaN,0.0,0,ИФ>5ВСТ_ДЕШ,2.59,NaN,NaN,NaN,NaN
1,ТОВ0121162,01Ш210012,1091,ДЕШЕВАЯ,"Брошь, фианит",NaN,0.0,0,ИФ>5ВСТ_ДЕШ,2.94,NaN,NaN,NaN,NaN
2,ТОВ0121163,01Ш210017,1091,ДЕШЕВАЯ,"Брошь, фианит",NaN,0.0,0,ИФ>5ВСТ_ДЕШ,3.16,NaN,NaN,NaN,NaN
3,ТОВ0121176,068826-1,1091,ДЕШЕВАЯ,"Брошь, фианит",NaN,0.0,0,ИФ>5ВСТ_ДЕШ,2.94,NaN,NaN,NaN,NaN
4,ТОВ0121181,07Ш115232,1091,ДЕШЕВАЯ,"Брошь, фианит",NaN,0.0,0,ИФ>5ВСТ_ДЕШ,3.16,NaN,NaN,NaN,NaN
5,ТОВ0121201,1Б6004Я,1093,ДОРОГАЯ,"Брошь, фианит",NaN,0.0,0,ИФ>5ВСТ_СР/ДОР,6.14,NaN,NaN,NaN,NaN
6,ТОВ0121275,ЕКВ-09,1091,ДЕШЕВАЯ,"Брошь, фианит",NaN,0.0,0,ИФ>5ВСТ_ДЕШ,2.77,NaN,NaN,NaN,NaN
7,ТОВ0121277,ЕКВ-13,1092,СРЕДНЯЯ,"Брошь, фианит",NaN,0.0,0,ИФ>5ВСТ_СР/ДОР,3.27,NaN,NaN,NaN,NaN
8,ТОВ0121285,ЛИ041023-2,1091,ДЕШЕВАЯ,"Брошь, фианит",NaN,0.0,0,ИФ>5ВСТ_ДЕШ,3.03,NaN,NaN,NaN,NaN
9,ТОВ0121288,ЛИ095771-4,1092,СРЕДНЯЯ,"Брошь, фианит",NaN,0.0,0,ИФ>5ВСТ_СР/ДОР,3.20,NaN,NaN,NaN,NaN
